In [2]:
import chess.pgn
import sys

f1 = open('white_wins.txt', 'w')
f2 = open('black_wins.txt', 'w')
pgn = open("train.pgn")
n_games = 0

while True:
    game = chess.pgn.read_game(pgn)
    try:
        result = game.headers["Result"]
    except:
        break

    if result == "1-0":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f1.write(str(board))
            f1.write("\n- - - - - - - -\n")
        f1.write("\n= = = = = = = =\n")
    if result == "0-1":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f2.write(str(board))
            f2.write("\n- - - - - - - -\n")
        f2.write("\n= = = = = = = =\n")

f1.close()        
f2.close()
pgn.close()
print("Done. ", n_games, " games parsed")

ModuleNotFoundError: No module named 'chess'

In [2]:
import tensorflow as tf
white = open('white_wins.txt', 'r')
black = open('black_wins.txt', 'r')



In [3]:
import numpy as np

# convert all data to a list form 
i = 0
boards = []
for line in white:
    temp = line.strip().split(" ")
    boards.append(temp)
# tst = np.load("A00-139_first_10000.npz")
# boards = tst['arr_0']
# targets = tst['arr_1']
# print(boards.shape)
# print(boards[0,1,:,:])
# print(targets.shape)
# print(boards)
boards_data = np.asarray(boards)

black_boards = []
for line in black:
    temp = line.strip().split(" ")
    black_boards.append(temp)


In [4]:
# First lets create a 8x8 boards
i = 0
boards_final = []
checklist = ['r','n', 'b', 'q', 'k', 'p', 'R','N', 'B', 'Q', 'K', 'P', '.']

for i in range(len(boards_data)):
    if any(check in boards_data[i] for check in checklist):
        boards_final.append(boards_data[i])
    
# print(len(boards_final))
# print(boards_final)
bbf = []
bb = np.asarray(black_boards)
for i in range(len(bb)):
    if any(check in bb[i] for check in checklist):
        bbf.append(bb[i])

In [5]:
# restructure the white boards to a chess board shape
boards_np = np.asarray(boards_final)
boards_np = boards_np.reshape((int(boards_np.shape[0]/8),8,8))

# restructure the black boards data to a chess board shape

bbf_np = np.asarray(bbf)
bbf_np = bbf_np.reshape((int(bbf_np.shape[0]/8),8,8))

# print(boards_np.shape)

In [6]:
# print(boards_np[0])
# convert to a map of 6x8x8 data type with the extra channel for 
# mapping the 6 types of chess pieces
boards_res = []

for i in range(len(boards_np)):
    test = boards_np[i]
    temp1 = np.zeros(test.shape)
    temp2 = np.zeros(test.shape)
    temp3 = np.zeros(test.shape)
    temp4 = np.zeros(test.shape)
    temp5 = np.zeros(test.shape)
    temp6 = np.zeros(test.shape)
    temp = []
    for i in range(len(test)):
        for j in range(len(test[0])):
            # for the rooks
            if test[i][j] == 'r':
                temp1[i][j] = -1
            if test[i][j] == 'R':
                temp1[i][j] = 1

            #for the n's
            if test[i][j] == 'n':
                temp2[i,j] = -1
            if test[i,j] == 'N':
                temp2[i,j] = 1

             #for the bishop
            if test[i][j] == 'b':
                temp3[i,j] = -1
            if test[i,j] == 'B':
                temp3[i,j] = 1

             #for the queen
            if test[i,j] == 'q':
                temp4[i,j] = -1
            if test[i,j] == 'Q':
                temp4[i,j] = 1

             #for the king
            if test[i][j] == 'k':
                temp5[i,j] = -1
            if test[i,j] == 'K':
                temp5[i,j] = 1

             #for the pawns
            if test[i][j] == 'p':
                temp6[i,j] = -1
            if test[i,j] == 'P':
                temp6[i,j] = 1

    temp.append(temp1)
    temp.append(temp2)
    temp.append(temp3)
    temp.append(temp4)
    temp.append(temp5)
    temp.append(temp6)
    temp = np.asarray(temp)
    boards_res.append(temp)

    
boards_res = np.asarray(boards_res)
print(boards_res.shape)
# print(boards_res)
# print(temp1)
# print(temp2)
# print(temp3)
# print(temp4)
# print(temp5)
# print(temp6)

(216982, 6, 8, 8)


In [7]:
black_res = []

for i in range(len(bbf_np)):
    test = bbf_np[i]
    temp1 = np.zeros(test.shape)
    temp2 = np.zeros(test.shape)
    temp3 = np.zeros(test.shape)
    temp4 = np.zeros(test.shape)
    temp5 = np.zeros(test.shape)
    temp6 = np.zeros(test.shape)
    temp = []
    for i in range(len(test)):
        for j in range(len(test[0])):
            # for the rooks
            if test[i][j] == 'r':
                temp1[i][j] = -1
            if test[i][j] == 'R':
                temp1[i][j] = 1

            #for the n's
            if test[i][j] == 'n':
                temp2[i,j] = -1
            if test[i,j] == 'N':
                temp2[i,j] = 1

             #for the bishop
            if test[i][j] == 'b':
                temp3[i,j] = -1
            if test[i,j] == 'B':
                temp3[i,j] = 1

             #for the queen
            if test[i,j] == 'q':
                temp4[i,j] = -1
            if test[i,j] == 'Q':
                temp4[i,j] = 1

             #for the king
            if test[i][j] == 'k':
                temp5[i,j] = -1
            if test[i,j] == 'K':
                temp5[i,j] = 1

             #for the pawns
            if test[i][j] == 'p':
                temp6[i,j] = -1
            if test[i,j] == 'P':
                temp6[i,j] = 1

    temp.append(temp1)
    temp.append(temp2)
    temp.append(temp3)
    temp.append(temp4)
    temp.append(temp5)
    temp.append(temp6)
    temp = np.asarray(temp)
    black_res.append(temp)

    
black_res = np.asarray(black_res)
print(black_res.shape)
# print(black_res)

(201837, 6, 8, 8)


In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# NOTE: maybe try training on complete games instead of single board positions
# NOTE: (-1, -1, 6, 8, 8)


# boards_res is of the form (216982,6,8,8) which is the correct shape for white wins
white_input_data = boards_res # board position where white wins
black_input_data = black_res # board position where black wins
white_input_label = np.ones(boards_res.shape[0])
black_input_label = (np.zeros(black_res.shape[0]))
print(white_input_label.shape)
print(black_input_label.shape)
input_label = np.int32(np.concatenate((white_input_label, black_input_label)))
print(input_label.shape)
# print(input_label)
input_data = np.float32(np.concatenate((white_input_data, black_input_data)))
print(input_data.shape)
# print(type(input_data[0][0][0][0]))

inp_data, eval_data, inp_label, eval_label = train_test_split(input_data, input_label, test_size=0.33, random_state=42)

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_fn(features, labels, mode):
    # Convolution layer #1
    input_layer = tf.reshape(features, [-1,6,8,8])
    conv1 = tf.layers.conv2d(inputs=input_layer, 
                             filters=64,
                             kernel_size=[5,5],
                             padding="same",
                             activation=tf.nn.relu)
#     print(conv1.shape)
    # Pooling layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
    dropout1 = tf.layers.dropout(inputs=pool1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)

#     print(pool1.shape)
    # print(type(pool1))

    # Convolution layer #2
    conv2 = tf.layers.conv2d(inputs=dropout1,
                            filters=32,
                            kernel_size=[3,3],
                            padding="same",
                            activation=tf.nn.relu)
#     print(conv2.shape)
    # print(type(conv2))
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    
    dropout2 = tf.layers.dropout(inputs=pool2, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
    
#     print(dropout2.shape)
    
    # Dense Layer
    pool2_flat = tf.reshape(dropout2, [-1, 1*2*32])
    dense = tf.layers.dense(inputs=pool2_flat, units=256, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)

#     print("Dense:", dense.shape)
    # Logit layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
#     print("Logits:", logits.shape)
    # print(logits)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

def main(unused_argv):
    
    for times in range(5):
        inp_data, eval_data, inp_label, eval_label = train_test_split(input_data, input_label, test_size=0.33, random_state=42)
        # restructure input data 
        labels = input_label
        # setting the mode to training right now
        # mode = tf.estimator.ModeKeys.TRAIN

        cnn_classifier = tf.estimator.Estimator(model_fn=cnn_fn, model_dir="cnn_model")

        tensor_to_log = {"probabilites":"softmax_tensor"}
        logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=2500)

        # batch_size = 1000 takes about 280 sec (a little too much for my computer)
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x=inp_data,
            y=inp_label,
            batch_size=100,
            num_epochs=None,
            shuffle=True)

        cnn_classifier.train(input_fn=train_input_fn,
                              steps=5000,
                              hooks=[logging_hook])

        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x=eval_data,
            y=eval_label,
            num_epochs=1,
            shuffle=False)
        eval_results = cnn_classifier.evaluate(input_fn=eval_input_fn)
        print(eval_results)

if __name__ == "__main__":
    tf.app.run()

# cnn_fn(input_data, input_label, mode)


(216982,)
(201837,)
(418819,)
(418819, 6, 8, 8)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3b21c2fa20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn_model/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Savi

INFO:tensorflow:loss = 0.46635, step = 42501 (1.009 sec)
INFO:tensorflow:global_step/sec: 109.842
INFO:tensorflow:loss = 0.556694, step = 42601 (0.907 sec)
INFO:tensorflow:global_step/sec: 113.839
INFO:tensorflow:loss = 0.54367, step = 42701 (0.878 sec)
INFO:tensorflow:global_step/sec: 112.629
INFO:tensorflow:loss = 0.539539, step = 42801 (0.888 sec)
INFO:tensorflow:global_step/sec: 112.257
INFO:tensorflow:loss = 0.450457, step = 42901 (0.890 sec)
INFO:tensorflow:global_step/sec: 110.945
INFO:tensorflow:loss = 0.456703, step = 43001 (0.901 sec)
INFO:tensorflow:global_step/sec: 111.404
INFO:tensorflow:loss = 0.458815, step = 43101 (0.898 sec)
INFO:tensorflow:global_step/sec: 110.941
INFO:tensorflow:loss = 0.467193, step = 43201 (0.901 sec)
INFO:tensorflow:global_step/sec: 110.206
INFO:tensorflow:loss = 0.48637, step = 43301 (0.907 sec)
INFO:tensorflow:global_step/sec: 99.3557
INFO:tensorflow:loss = 0.458266, step = 43401 (1.007 sec)
INFO:tensorflow:global_step/sec: 101.617
INFO:tensorfl

INFO:tensorflow:loss = 0.465827, step = 46401 (1.514 sec)
INFO:tensorflow:global_step/sec: 62.4974
INFO:tensorflow:loss = 0.508141, step = 46501 (1.600 sec)
INFO:tensorflow:global_step/sec: 70.7214
INFO:tensorflow:loss = 0.490677, step = 46601 (1.416 sec)
INFO:tensorflow:global_step/sec: 77.52
INFO:tensorflow:loss = 0.563786, step = 46701 (1.288 sec)
INFO:tensorflow:global_step/sec: 83.2197
INFO:tensorflow:loss = 0.424272, step = 46801 (1.202 sec)
INFO:tensorflow:global_step/sec: 66.4176
INFO:tensorflow:loss = 0.415937, step = 46901 (1.506 sec)
INFO:tensorflow:global_step/sec: 70.5654
INFO:tensorflow:loss = 0.552592, step = 47001 (1.418 sec)
INFO:tensorflow:global_step/sec: 70.3281
INFO:tensorflow:loss = 0.479832, step = 47101 (1.421 sec)
INFO:tensorflow:global_step/sec: 73.4877
INFO:tensorflow:loss = 0.483021, step = 47201 (1.362 sec)
INFO:tensorflow:global_step/sec: 71.6841
INFO:tensorflow:loss = 0.410954, step = 47301 (1.395 sec)
INFO:tensorflow:global_step/sec: 74.6075
INFO:tensorf

INFO:tensorflow:loss = 0.434728, step = 50001
INFO:tensorflow:global_step/sec: 92.7006
INFO:tensorflow:loss = 0.528395, step = 50101 (1.075 sec)
INFO:tensorflow:global_step/sec: 100.151
INFO:tensorflow:loss = 0.435732, step = 50201 (0.998 sec)
INFO:tensorflow:global_step/sec: 98.4551
INFO:tensorflow:loss = 0.610151, step = 50301 (1.016 sec)
INFO:tensorflow:global_step/sec: 98.5974
INFO:tensorflow:loss = 0.398554, step = 50401 (1.014 sec)
INFO:tensorflow:global_step/sec: 100.333
INFO:tensorflow:loss = 0.470167, step = 50501 (0.998 sec)
INFO:tensorflow:global_step/sec: 97.5266
INFO:tensorflow:loss = 0.435499, step = 50601 (1.024 sec)
INFO:tensorflow:global_step/sec: 97.4469
INFO:tensorflow:loss = 0.451468, step = 50701 (1.026 sec)
INFO:tensorflow:global_step/sec: 90.8509
INFO:tensorflow:loss = 0.522448, step = 50801 (1.101 sec)
INFO:tensorflow:global_step/sec: 83.083
INFO:tensorflow:loss = 0.457619, step = 50901 (1.203 sec)
INFO:tensorflow:global_step/sec: 91.3858
INFO:tensorflow:loss = 

{'accuracy': 0.79080534, 'loss': 0.42621377, 'global_step': 55000}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3b0338bcc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn_model/model.ckpt-55000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I

INFO:tensorflow:loss = 0.46304, step = 57501 (0.979 sec)
INFO:tensorflow:global_step/sec: 110.951
INFO:tensorflow:loss = 0.447968, step = 57601 (0.897 sec)
INFO:tensorflow:global_step/sec: 113.178
INFO:tensorflow:loss = 0.407538, step = 57701 (0.884 sec)
INFO:tensorflow:global_step/sec: 107.116
INFO:tensorflow:loss = 0.447823, step = 57801 (0.933 sec)
INFO:tensorflow:global_step/sec: 110.279
INFO:tensorflow:loss = 0.472732, step = 57901 (0.907 sec)
INFO:tensorflow:global_step/sec: 110.711
INFO:tensorflow:loss = 0.445159, step = 58001 (0.903 sec)
INFO:tensorflow:global_step/sec: 104.778
INFO:tensorflow:loss = 0.482717, step = 58101 (0.954 sec)
INFO:tensorflow:global_step/sec: 111.545
INFO:tensorflow:loss = 0.369863, step = 58201 (0.896 sec)
INFO:tensorflow:global_step/sec: 106.332
INFO:tensorflow:loss = 0.476308, step = 58301 (0.941 sec)
INFO:tensorflow:global_step/sec: 114.5
INFO:tensorflow:loss = 0.435701, step = 58401 (0.873 sec)
INFO:tensorflow:global_step/sec: 112.6
INFO:tensorflow

INFO:tensorflow:loss = 0.500967, step = 61401 (0.973 sec)
INFO:tensorflow:global_step/sec: 111.029
INFO:tensorflow:loss = 0.434338, step = 61501 (0.899 sec)
INFO:tensorflow:global_step/sec: 112.731
INFO:tensorflow:loss = 0.419786, step = 61601 (0.887 sec)
INFO:tensorflow:global_step/sec: 110.699
INFO:tensorflow:loss = 0.384741, step = 61701 (0.904 sec)
INFO:tensorflow:global_step/sec: 106.441
INFO:tensorflow:loss = 0.509649, step = 61801 (0.939 sec)
INFO:tensorflow:global_step/sec: 74.1429
INFO:tensorflow:loss = 0.513273, step = 61901 (1.349 sec)
INFO:tensorflow:global_step/sec: 89.853
INFO:tensorflow:loss = 0.474155, step = 62001 (1.112 sec)
INFO:tensorflow:global_step/sec: 88.7805
INFO:tensorflow:loss = 0.372985, step = 62101 (1.126 sec)
INFO:tensorflow:global_step/sec: 107.407
INFO:tensorflow:loss = 0.480476, step = 62201 (0.931 sec)
INFO:tensorflow:global_step/sec: 114.263
INFO:tensorflow:loss = 0.460366, step = 62301 (0.875 sec)
INFO:tensorflow:global_step/sec: 113.509
INFO:tensor

SystemExit: 

/home/sbansal6/my-envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
